# Esplorazione dataset progetto AWS

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [ ]:
df_bitcoin = pd.read_csv("/content/BTC_EUR_Historical_Data.csv")
df_bitcoin_google_trend = pd.read_csv("/content/google_trend_bitcoin.csv")

In [ ]:
df_XMR = pd.read_csv("/content/XMR_EUR Kraken Historical Data.csv")
df_XMR_google_trend = pd.read_csv("/content/google_trend_monero.csv")

In [ ]:
df_bitcoin.info()

# Abbiamo 1887 entries senza valori mancanti con tutti i dati storati come stringhe.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1887 entries, 0 to 1886
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      1887 non-null   object
 1   Price     1887 non-null   object
 2   Open      1887 non-null   object
 3   High      1887 non-null   object
 4   Low       1887 non-null   object
 5   Vol.      1887 non-null   object
 6   Change %  1887 non-null   object
dtypes: object(7)
memory usage: 103.3+ KB


In [ ]:
df_bitcoin

,Date,Price,Open,High,Low,Vol.,Change %
0,03/12/2024,"65,619.5","65,860.0","66,135.3","65,200.0",0.19K,-0.37%
1,03/11/2024,"65,859.9","63,074.2","66,600.0","61,517.2",1.88K,4.42%
2,03/10/2024,"63,074.3","62,613.1","63,930.6","62,400.0",0.72K,0.68%
3,03/09/2024,"62,649.9","62,425.2","62,781.1","62,124.5",0.37K,0.38%
4,03/08/2024,"62,413.8","61,130.1","63,999.0","60,483.8",1.61K,2.10%
...,...,...,...,...,...,...,...
1882,01/16/2019,"3,164.7","3,149.2","3,228.9","3,140.2",5.52K,0.49%
1883,01/15/2019,"3,149.2","3,197.6","3,214.3","3,119.8",5.36K,-1.51%
1884,01/14/2019,"3,197.6","3,075.0","3,238.8","3,068.1",7.29K,4.02%
1885,01/13/2019,"3,074.0","3,168.1","3,188.0","3,052.2",3.52K,-2.97%


In [ ]:
df_XMR.tail(20)

,Date,Price,Open,High,Low,Vol.,Change %
1878,01/20/2019,38.27,40.27,40.45,37.98,4.74K,-4.97%
1879,01/19/2019,40.27,39.33,40.84,39.33,2.25K,2.39%
1880,01/18/2019,39.33,39.76,39.78,38.90,3.28K,-1.08%
1881,01/17/2019,39.76,40.26,40.40,38.73,5.31K,-1.24%
1882,01/16/2019,40.26,38.93,40.42,38.85,2.41K,3.28%
1883,01/15/2019,38.98,39.72,40.49,38.50,6.43K,-1.86%
1884,01/14/2019,39.72,37.18,40.47,37.18,6.31K,6.83%
1885,01/13/2019,37.18,38.90,39.43,37.01,6.84K,-4.42%
1886,01/12/2019,38.90,39.52,39.67,38.57,2.62K,-1.19%
1887,01/11/2019,39.37,39.12,40.50,38.47,5.23K,0.64%


In [ ]:
df_XMR_google_trend

,Settimana,Monero_interesse
0,2019-03-17,26
1,2019-03-24,23
2,2019-03-31,28
3,2019-04-07,25
4,2019-04-14,23
...,...,...
255,2024-02-04,50
256,2024-02-11,32
257,2024-02-18,29
258,2024-02-25,32


# 1* Pipeline

In [ ]:
def clean_dataset(df: pd.DataFrame) -> pd.DataFrame:
  """Cleans and normalize the dataset.

  This function is tailored  for a dataset with the following
  columns: `["Date", "Price", "Open", "High", "Low", "Vol.", "Change %"]`.


  Some assumptions on the origin file:
  * `Date` is a string in US format **%m/%d/%Y** (es. "03/25/2025").
  *  Numerical columns could cointain separators like (`,`).
  * `Vol.` it can have suffixes like **K/M/B** (es. "3.39K", "1.2M").
  * `Change %` includes the symbol `%` (es. "2.15%"), which we remove:
    the result is in percentages point (es. 2.15, with no 0.0215).

  Effects:
  - Converts "Date" in "dateime64[ns]"
  - Converts "Price","Open","High","Low" in "float64"
  - Converts "Vol." in full number
  - Converts "Change %" in float (percentages points)


  """

  for col in ["Price","Open","High","Low","Vol.","Change %"]:

    if df[col].dtype in ["object","string"]:

      # Tolgo le virgole cosi posso trasformare in float poi
      df[col] = df[col].str.replace(",", "", regex=False)

      # Tolgo K e %
      df[col] = df[col].str.replace("%|K|M|B", "", regex=True)

    else: None

  # Lascia solo numeri e /
  df["Date"] = df["Date"].str.replace(r"[-.]", "/", regex=True)

  # Trasforma in datetime la colonna Date
  df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

  # cambia formato Date in giorno/mese/anno
  df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")

  # Conversione in datetime di Date
  df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

  # Conversione di tutte le colonne rimaste in float
  df = df.astype({"Price": float, "Open": float, "High": float, "Low": float, "Change %": float, "Vol.": float })

  # moltiplico Vol. x 1000 cosi da avere il numero corretto
  df["Vol."] = df["Vol."] * 1000

  return df


In [ ]:
def imputer_dataset(df: pd.DataFrame) -> pd.DataFrame:
  """
  1.Riempie i valori -1 nella colonna 'Price' calcolandoli
  dal prezzo della riga successiva e dalla variazione in 'Change %'.

  Formula:
        Price_oggi = Price_domani * (1 + Change% / 100)

  2. Riempie i NaN in 'Vol.' usando KNNImputer.


  Assunzioni:
    - DataFrame ordinato in data decrescente (prima riga = giorno più recente).
    - 'Change %' è espresso in punti percentuali (es. -1.05 per -1,05%).

  """

  df = df.copy()

  # Conversione sicura a numerico
  df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
  df["Change %"] = pd.to_numeric(df["Change %"], errors="coerce")

  # Maschera per i valori da riempire
  mask = df["Price"] == -1

  # Prezzo della riga successiva (nel tempo)
  prezzo_successivo = df["Price"].shift(-1)

  # Calcolo prezzo stimato
  prezzo_calcolato = (prezzo_successivo * (1 + df["Change %"] / 100)).round(1)

  # Sostituzione solo dove serve
  df.loc[mask, "Price"] = prezzo_calcolato[mask]

  # KNNImputer solo per Vol.
  imputer = KNNImputer(n_neighbors=14, weights="distance")
  vol_df = df[["Vol."]]  # DataFrame 2D
  vol_imputed = imputer.fit_transform(vol_df)
  df["Vol."] = vol_imputed.round(0)


  return df

# SALVATAGGIO FILE IN FORMATO PARQUET ED ESPLORAZIONE

In [ ]:
# Se si esegue la seconda volta dà errore
df_bitcoin_cleaned = clean_dataset(df_bitcoin)

In [ ]:
df_bitcoin_cleaned_imputed = imputer_dataset(df_bitcoin_cleaned)

In [ ]:
df_bitcoin_cleaned_imputed

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-12,65619.5,65860.0,66135.3,65200.0,190.0,-0.37
1,2024-03-11,65859.9,63074.2,66600.0,61517.2,1880.0,4.42
2,2024-03-10,63074.3,62613.1,63930.6,62400.0,720.0,0.68
3,2024-03-09,62649.9,62425.2,62781.1,62124.5,370.0,0.38
4,2024-03-08,62413.8,61130.1,63999.0,60483.8,1610.0,2.10
...,...,...,...,...,...,...,...
1882,2019-01-16,3164.7,3149.2,3228.9,3140.2,5520.0,0.49
1883,2019-01-15,3149.2,3197.6,3214.3,3119.8,5360.0,-1.51
1884,2019-01-14,3197.6,3075.0,3238.8,3068.1,7290.0,4.02
1885,2019-01-13,3074.0,3168.1,3188.0,3052.2,3520.0,-2.97


I file Parquet hanno i seguenti vantaggi :

- È binario → più compatto e veloce da leggere/scrivere rispetto al CSV.

- Supporta compressione (compression='snappy', 'gzip', ecc.).

- Mantiene i tipi di dato (mentre il CSV li converte tutti in testo (string o object)).

In [ ]:
df_bitcoin_cleaned_imputed.to_parquet("dati_bitcoin.parquet", engine="pyarrow", index=False)

In [ ]:
df_bitcoin_parquet = pd.read_parquet("/content/dati_bitcoin.parquet", engine="pyarrow")